---
title: "粒子フィルターの実装 | Particles Package"
subtitle: "`NumPy` と `SciPy` で粒子フィルターを実装する"
author: "司馬博文"
date: 12/11/2023
categories: [Particles, Python]
image: sample_plot.png
bibliography: ../../../assets/2023.bib
csl: ../../../assets/apalike.csl
abstract-title: 概要
abstract: Pythonを用いて粒子フィルターを実装する方法を，Nicolas Chopinによるparticlesパッケージを参考に解説する．
# jupyter: genai
---



[Nicolas Chopin](https://nchopin.github.io/) による逐次モンテカルロ法のための Python パッケージ [`particles`](https://particles-sequential-monte-carlo-in-python.readthedocs.io/en/latest/index.html) の実装を参考に，`NumPy`, `SciPy` のみを用いて1から粒子フィルターを実装することで，その仕組みを理解することを目指す．

## リサンプリングの実装

まずリサンプリング法を実装する．今回は系統的リサンプリング法 [@Carpenter+1999] を用いることとする．^[実はこのリサンプリング法は [@Kitagawa1996] の付録で原型が（全く決定論的なアルゴリズムとして）提案されている]

### システマティックリサンプリング {#sec-systematic}

系統的リサンプリングとは，粒子数 $N$ から $M$ 個のサンプルを復元抽出する方法であって，次の2段階からなる．


::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\mathrm{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\mathtt{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



1. 各区間 $\Square{\frac{n-1}{M},\frac{n}{M}}\subset[0,1]\;(n\in[M])$ での変動の由来をただ一つのサンプル $U\sim\rU([0,1])$ から取ってしまい，
$$
U^{(n)}:=\frac{n-1+U}{N}\quad(n\in[M])
$$
と乱数列を定める．
2. 正規化荷重 $\{w^{(n)}\}_{n=1}^N$ が定める累積和
$$
F(n):=\sum_{i=1}^nw^{(i)}
$$
に対して，この一般化逆関数 $F^-:[0,1]\to[N]$ を通じて，$A^{n}:=F^{-1}(U^{(n)})\;(n\in[M])$ をサンプルとする．

これにより，粒子の添字の対応 $(1,\cdots,N)\mapsto A^{1:M}$ が得られる．


In [ ]:
import numpy as np
from numba import jit

@jit(nopython=True)
def inverse_cdf(su, W):
    """Inverse CDF algorithm for a finite distribution.
    su: (M,) ndarray of sorted uniform variables
    W: (N,) ndarray of normalized weights"""
    j = 0
    s = W[0]
    M = su.shape[0]
    A = np.empty(M, dtype=np.int64)
    for n in range(M):
        while su[n] > s:
            j += 1
            s += W[j]
        A[n] = j
    return A

def systematic(W, M):
    """Systematic resampling
    W: (N,) ndarray of normalized weights
    M : number of resampled points"""
    su = (random.rand(1) + np.arange(M)) / M
    return inverse_cdf(su, W)

### 荷重を保持する`Weights`クラス

次に，`SMC` に繋げるために，粒子の荷重を保持するためのクラスを定義する．粒子の荷重は極めて小さくなり得るため，対数によって保持する．このクラス内の属性として，正規化荷重もESSも得られるようにする：

* `lw`：正規化されていない荷重を対数で保持
* `w`：正規化された荷重
* `ESS`：有効サンプル数^[有効サンプル数の定義については [@Chopin-Papaspiliopoulos20-SMC] 参照．]

これらの属性を `__init__(lw)` 内で計算する．加えて `add(delta)` メソッドで，incremental weightsを乗じるルーチンを用意する．


In [ ]:
class Weights:
    """A class to hold the N weights of the particles"""
    def __init__(self, lw=None):
        self.lw = lw  # t=0で呼ばれた際はNoneである
        if lw is not None:
            self.lw[np.isnan(self.lw)] = -np.inf  # 欠損値処理
            m = self.lw.max()
            w = np.exp(self.lw - m)  # 大きすぎる値にならないように
            s = w.sum()
            self.W = w / s  # 正規化荷重
            self.ESS = 1.0 / np.sum(self.W ** 2)
            self.log_mean = m + np.log(s / self.N)
    
    @property
    def N(self):
        """Number of particles"""
        return 0 if self.lw is None else self.lw.shape[0]

    def add(self, delta):
        """Add increment weights delta to the log weights"""
        if self.lw is None:
            return self.__class__(lw=delta)
        else:
            return self.__class__(lw=self.lw + delta)

初期化は
$$
W^i=\frac{e^{\log w^i-m}}{\sum_{j=1}^Ne^{\log w^j-m}}
$$
$$
m:=\log\paren{\max_{i\in[N]}w^i}
$$
に基づいて計算されている．`log_mean` は
$$
\begin{align*}
    &\log\paren{\max_{i\in[N]}w^i}+\log\paren{\frac{\sum_{j=1}^Ne^{\log w^j-m}}{N}}\\
    &=\log\paren{\frac{1}{N}\sum_{j=1}^Nw^j}
\end{align*}
$$
という値である．

## 粒子の情報保持：`ParticleHistory`クラス

### 情報を収集する`Collector`クラス

SMCの結果をプロットするために，各時間で粒子の標本統計量を `SMC` クラス（ @sec-SMC ）から適宜抜き出して保存しておくためのクラス `Summaries` を作成する．抜き出すためのメソッドを `Collector` クラスの継承クラスとして定義する．


In [ ]:
class Collector:
    """Base class for collectors"""
    def __init__(self, **kwargs):
        self.summary = []

    def collect(self, smc):
        self.summary.append(self.fetch(smc))

class ESSs(Collector):
    summary_name = "ESSs"
    def fetch(self, smc):
        return smc.wgts.ESS

class LogLts(Collector):
    summary_name = "LogLts"
    def fetch(self, smc):
        return smc.logLt

class Rs_flags(Collector):
    summary_name = "Rs_flags"
    def fetch(self, smc):
        return smc.rs_flag

class Moments(Collector):
    """Collects empirical moments of the particles"""
    summary_name = "Moments"
    def fetch(self, smc):
        m = np.average(smc.X, weights=smc.wgts.W, axis=0)
        m2 = np.average(smc.X ** 2, weights=smc.wgts.W, axis=0)
        v = m2 - m ** 2
        return {"mean": m, "var": v}

default_collector_cls = [ESSs, LogLts, Rs_flags]

### 標本統計量を保持する`Summaries`クラス

このクラスはデフォルトで用意されている `default_collector_cls` に加えて，`cols`引数で指定されたメソッドを追加し，`collect()` メソッドが呼ばれるとこれらを集めて属性として保持する．


In [ ]:
class Summaries:
    """A class to hold the summaries of the SMC algorithm"""
    def __init__(self, cols):
        self._collectors = [cls() for cls in default_collector_cls]
        if cols is not None:
            self._collectors.extend(col() for col in cols)
        for col in self._collectors:
            setattr(self, col.summary_name, col.summary)

    def collect(self, smc):
        for col in self._collectors:
            col.collect(smc)

### ヒストリを保持する`ParticleHistory`クラス

[`deque`オブジェクト](https://docs.python.org/ja/3/library/collections.html#collections.deque) としてヒストリを格納するためのクラス`ParticleHistory`実装する．これにより直前 $k$ ステップの情報だけを保持出来るように作れるが，今回はプロットのために全履歴を保持する．


In [ ]:
class ParticleHistory:
    """History of the particles
    Full history that keeps all the particle systems based on lists.
    """
    def __init__(self, fk):
        self.X, self.A, self.wgts = [], [], []
        self.fk = fk

    def save(self, smc):
        self.X.append(smc.X)
        self.A.append(smc.A)
        self.wgts.append(smc.wgts)

In [ ]:
def generate_hist_obj(option, smc):
    if option is True:
        return ParticleHistory(smc.fk)
    else:
        return None

## 実行部分：`SMC`クラス {#sec-SMC}

このクラスがやるべきことは多い．Feynman-Kacモデル `fk`（ @sec-FeynmanKac で後述），粒子数 `N`，リサンプリング法 `resampling` を引数に取り，粒子フィルターを実行する．

最も大事なこととして，本クラスはイテレータとして定義し，`__next__` メソッドを実装する．そして `run()` メソッドで `__next__` を終了するまで繰り返し呼び出すことでイテレータプロトコルを実行する．

`__next__`メソッドでは，次のような処理を行う：

1. 終了フラッグ `fk.done(self)` が立っているかどうかを確認する．
2. $t=0$ の場合，最初の粒子を初期分布 $M_0$ から $N$ 個サンプリングする．
3. $t>0$ の場合は，リサンプリングと粒子移動を行う．これは `resample_move()` メソッドで行う．
   1. リサンプリングフラッグ `fk.time_to_resample(self)` が立っている場合にリサンプリングを `systematic` メソッド（ @sec-systematic ）により行う．これにより，移動（変異）する粒子 $A^{1:N}_t$ を確定させる．
   2. 確率核 $M_t(X_{t-1}^{A_t^{1:N}},-)$ に従って，粒子 $X_t^{1:N}$ をサンプリングする．
4. 粒子の荷重を更新する．これは `reweight_particles()` メソッドで行う．
5. `compute_summaries`メソッドを呼び出して，粒子の標本統計量を `Summaries` クラスに，ヒストリを `Particle History` クラスに追記する．
6. 時刻 $t$ を進めて 3.に戻る．


In [ ]:
class SMC:
    """Metaclass for SMC algorithms"""

    def __init__(
        self,
        fk=None,
        N=100,
        resampling="systematic",
        ESSrmin=0.5,
        store_history=False,
        collect=None,
    ):

        self.fk = fk
        self.N = N
        self.resampling = resampling
        self.ESSrmin = ESSrmin

        # initialisation
        self.t = 0
        self.rs_flag = False  # no resampling at time 0, by construction
        self.logLt = 0.0
        self.wgts = Weights()
        self.X, self.Xp, self.A = None, None, None

        self.summaries = Summaries(collect)
        self.hist = generate_hist_obj(store_history, self)

    def generate_particles(self):
        """Generate particles at time t=0"""
        self.X = self.fk.M0(self.N)
    
    def reset_weights(self):
        """Reset weights to uniform after a resamping step"""
        self.wgts = Weights()
    
    def resample_move(self):
        """Adaptively resample and move particles at time t"""
        self.rs_flag = self.fk.time_to_resample(self)
        if self.rs_flag:
            self.A  = systematic(self.wgts.W, M=self.N)
            self.Xp = self.X[self.A]
            self.reset_weights()
        else:
            self.A = np.arange(self.N)
            self.Xp = self.X
        self.X = self.fk.M(self.t, self.Xp)

    def reweight_particles(self):
        """Reweight particles at time t"""
        self.wgts = self.wgts.add(self.fk.logG(self.t, self.Xp, self.X))

    def compute_summaries(self):
        """Compute summaries at time t"""
        if self.t > 0:  # なぜかこれを前におかないとUnboundLocalErrorが出る
            prec_log_mean_w = self.log_mean_w
        self.log_mean_w = self.wgts.log_mean
        if self.t == 0 or self.rs_flag:
            self.loglt = self.log_mean_w
        else:
            self.loglt = self.log_mean_w - prec_log_mean_w
        self.logLt += self.loglt

        self.hist.save(self)
        self.summaries.collect(self)

    def __next__(self):
        """One step of the SMC algorithm"""
        if self.fk.done(self):
            raise StopIteration
        if self.t == 0:
            self.generate_particles()
        else:
            self.resample_move()
        self.reweight_particles()
        self.compute_summaries()
        self.t += 1

    def __iter__(self):
        return self

    def run(self):
        """Run the SMC algorithm until completion"""
        for _ in self:
            pass

## 粒子フィルタの実行：東京の年別気温データ

### Feynman-Kacモデルの枠組み {#sec-FeynmanKac}

`particle` パッケージの抽象クラス `FeynmanKac` は次のメソッドを持つ．^[Feynman-Kacモデルなどの用語については [@Chopin-Papaspiliopoulos20-SMC] 参照．]

* `M0(N)`: 初期分布 $M_0$ から $N$ 個のサンプルを生成する．
* `M(t, xp)`: カーネル $M_t(x_{t-1}|-)$ から $X_t$ をサイズ `xp.shape[0]` で生成する．
* `logG(t, xp, x)`: ポテンシャル $G_t(x_{t-1},x_t)$ の対数を返す．

加えて，粒子フィルターの実行時に必要なフラグも用意する．

* `time_to_resample(smc)`: `smc` オブジェクトを引数に取り，その属性 `smc.aux.ESS`, `smc.ESSrmin` からリサンプリングが必要かどうかを判定する．
* `done(smc)`: `smc` オブジェクトを引数に取り，その属性 `smc.t`, `smc.T` からアルゴリズムを終了すべきかどうかを判定する．

`particle` パッケージを使うときは `FeynmanKac` クラスを継承して用いることになるが，ここでは自分で定義していく．

### 使用するデータ

気象庁が [HP](https://www.data.jma.go.jp/obd/stats/etrn/view/annually_s.php?prec_no=44&block_no=47662&year=&month=&day=&view=) にて公開している1876年から2022年までの計147年分の東京の年別気温データを用いる．


In [ ]:
import pandas as pd

data = pd.read_csv("TemperatureDataAtTokyo.csv")
print(data.describe())

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(3.5, 3))

plt.title("temperature in Tokyo")
plt.xlabel("Year")
plt.ylabel("Temperature (Celsius)")

plt.scatter(data['年度'], data['日平均'], s=2)
plt.show()

### 気温の1次のトレンドモデル

気温の観測値 $\{y_k\}$ に対して，1次元の線型Gauss状態空間モデル
$$
\begin{cases}
x_k=x_{k-1}+v_k,\\
y_k=x_k+w_k.
\end{cases}
$$ {#eq-SSM}
$$
v_k\iidsim\rN(0,Q^2),\quad w_k\iidsim\rN(0,R^2),
$$
を想定する．このモデルを1次のトレンドモデルという [@北川2005-時系列解析入門 第11章]．

これを`SMC`メソッド（ @sec-SMC ）に渡せるように実装するには次のようにする：


In [ ]:
from numpy import random
from scipy import stats

class Bootstrap:
    """Abstract base class for Feynman-Kac models derived from State Space Model (1).
    """

    def __init__(self, data, T, R, Q):
        self.data = data
        self.T = T
        self.R = R
        self.Q = Q
    
    def M0(self, N):
        """Sample N times from initial distribution M0 of the FK model"""
        return random.normal(loc=13.6, scale=self.Q, size=N)
    
    def M(self, t, xp):  # xp: resampled previous state
        """Sample Xt from kernel Mt conditioned on Xt-1=xp"""
        return random.normal(loc=xp, scale=self.Q, size=xp.shape[0])
    
    def logG(self, t, xp, x):  # x: current state
        """Evaluate the log potential Gt(xt-1,xt)"""
        return stats.norm.logpdf(self.data[t], loc=x, scale=self.R)
    
    def time_to_resample(self, smc):
        """Return True if resampling is needed"""
        return smc.wgts.ESS < smc.N * smc.ESSrmin
    
    def done(self, smc):
        """Return True if the algorithm is done"""
        return smc.t >= self.T

### $(R,Q)=(0.2,0.1)$ の場合

仮に $(R,Q)=(0.2,0.1)$ としてみる．すなわち，システムノイズ $Q^2=0.1$ が小さく，観測ノイズ $R^2=0.4$ はそれよりは大きいとしている．


In [ ]:
model1 = Bootstrap(data=data['日平均'], T=data.shape[0], R=0.2, Q=0.1)
PF1 = SMC(fk=model1, N=1000, resampling="systematic", ESSrmin=0.5, collect=[Moments], store_history=True)
PF1.run()

In [ ]:
#| label: fig-2-1
#| fig-cap: (R,Q)=(0.2,0.1) の場合の粒子フィルターの実行結果
plt.figure(figsize=(3.5, 3))
plt.plot(data['日平均'], label='data', linestyle='', marker='.')
plt.plot([m['mean'] for m in PF1.summaries.Moments], label='filtered temperature trend')
plt.show()

少し揺らぎながらも，トレンドとして気温が上昇していく様子が見られる．

### $(R,Q)=(0.7,0.1)$ の場合

濾波して得たトレンドの揺らぎが少し大きいと思われたため，観測誤差はもう少し大きいものとして $(R,Q)=(0.7,0.1)$ としてみる．


In [ ]:
model4 = Bootstrap(data=data['日平均'], T=data.shape[0], R=0.7, Q=0.1)
PF4 = SMC(fk=model4, N=1000, resampling="systematic", ESSrmin=0.5, collect=[Moments], store_history=True)
PF4.run()

In [ ]:
#| fig-cap: "(R,Q)=(0.7,0.1) の場合の粒子フィルターの実行結果"
plt.figure(figsize=(3.5, 3))
plt.plot(data['日平均'], label='data', linestyle='', marker='.')
plt.plot([m['mean'] for m in PF4.summaries.Moments], label='filtered temperature trend')
plt.show()

こうしてトレンドとして少しばかり直線的なものが得られた．やはり上昇トレンドが見られる．

### $(R,Q)=(0.2,0.01)$ の場合

$Q^2=10^{-4}$ としてシステムノイズは極めて小さいと想定してみる．「トレンドは殆ど変化しない」という仮定を置いたことになる．


In [ ]:
model2 = Bootstrap(data=data['日平均'], T=data.shape[0], R=0.2, Q=0.01)
PF2 = SMC(fk=model2, N=1000, resampling="systematic", ESSrmin=0.5, collect=[Moments], store_history=True)
PF2.run()

In [ ]:
#| fig-cap: "(R,Q)=(0.2,0.01) の場合の粒子フィルターの実行結果"
plt.figure(figsize=(3.5, 3))
plt.plot(data['日平均'], label='data', linestyle='', marker='.')
plt.plot([m['mean'] for m in PF2.summaries.Moments], label='filtered temperature trend')
plt.show()

あまり良い当てはまりを見せないため，この気温の時系列を全てが観測誤差によるものだと理解するのは妥当ではないと考えられる．

### $(R,Q)=(0.2,1)$ の場合

逆にシステムノイズを極めて大きい値 $Q^2=1$ と設定する．トレンドは年別の揺らぎが大きいと想定したことになる．


In [ ]:
model3 = Bootstrap(data=data['日平均'], T=data.shape[0], R=0.2, Q=1.0)
PF3 = SMC(fk=model3, N=1000, resampling="systematic", ESSrmin=0.5, collect=[Moments], store_history=True)
PF3.run()

In [ ]:
#| fig-cap: "(R,Q)=(0.2,1) の場合の粒子フィルターの実行結果"
plt.figure(figsize=(3.5, 3))
plt.plot(data['日平均'], label='data', linestyle='', marker='.')
plt.plot([m['mean'] for m in PF3.summaries.Moments], label='filtered temperature trend')
plt.show()

とんでもない過適応を見せて，全てをトレンドとして説明してしまっており，これもまた妥当ではないと考えられる．

### カルマンフィルタとの比較

線型Gaussモデルを想定しているため，粒子フィルターは $N\to\infty$ の極限で最適フィルターであるカルマンフィルターに一致するはずである．そこで，`pykalman` パッケージを用いてこれを実装する．$(R,Q)=(0.2,0.1)$ とする．


In [ ]:
from pykalman import KalmanFilter
KF1 = KalmanFilter(initial_state_mean=13.6, initial_state_covariance=0.1,
                   transition_matrices=1, observation_matrices=1,
                   transition_covariance=0.1, observation_covariance=0.2, n_dim_state=1, n_dim_obs=1)
KF1 = KF1.em(data['日平均'], n_iter=5)  # EMアルゴリズムの過適応回避のため
(filtered_state_means, filtered_state_covariances) = KF1.filter(data['日平均'])

In [ ]:
#| fig-cap: "(R,Q)=(0.2,0.1) の場合のKalmanフィルターの実行結果"
plt.figure(figsize=(3.5, 3))
plt.plot(data['日平均'], label='data', linestyle='', marker='.')
plt.plot(filtered_state_means, label='filtered temperature trend')
plt.show()

たしかに @fig-2-1 と極めて似通った結果になっている．

### カルマン平滑化の結果


In [ ]:
(smoothed_state_means, smoothed_state_covariances) = KF1.smooth(data['日平均'])

In [ ]:
#| fig-cap: "(R,Q)=(0.2,0.1) の場合のKalman平滑化の実行結果"
plt.figure(figsize=(3.5, 3))
plt.plot(data['日平均'], label='data', linestyle='', marker='.')
plt.plot(smoothed_state_means, label='smoothed temperature trend')
plt.show()

より滑らかなトレンドが得られている．